#### Changing Data Types
* data types in spark: https://spark.apache.org/docs/latest/sql-ref-datatypes.html
* cast: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.cast.html?highlight=cast#pyspark.sql.Column.cast

In [0]:
# Reading in the countries.csv file and specifying the schema
countries_path = 'dbfs:/FileStore/read_write_data/countries.csv'
 
from pyspark.sql.types import IntegerType, StringType, DoubleType, StructField, StructType
countries_schema = StructType([
                    StructField("COUNTRY_ID", IntegerType(), False),
                    StructField("NAME", StringType(), False),
                    StructField("NATIONALITY", StringType(), False),
                    StructField("COUNTRY_CODE", StringType(), False),
                    StructField("ISO_ALPHA2", StringType(), False),
                    StructField("CAPITAL", StringType(), False),
                    StructField("POPULATION", DoubleType(), False),
                    StructField("AREA_KM2", IntegerType(), False),
                    StructField("REGION_ID", IntegerType(), True),
                    StructField("SUB_REGION_ID", IntegerType(), True),
                    StructField("INTERMEDIATE_REGION_ID", IntegerType(), True),
                    StructField("ORGANIZATION_REGION_ID", IntegerType(), True)
                    ]
                    )
 
countries=spark.read.csv(path=countries_path, header=True, schema=countries_schema)

In [0]:
# using .dtypes to view data types
countries.dtypes

Out[4]: [('COUNTRY_ID', 'int'),
 ('NAME', 'string'),
 ('NATIONALITY', 'string'),
 ('COUNTRY_CODE', 'string'),
 ('ISO_ALPHA2', 'string'),
 ('CAPITAL', 'string'),
 ('POPULATION', 'double'),
 ('AREA_KM2', 'int'),
 ('REGION_ID', 'int'),
 ('SUB_REGION_ID', 'int'),
 ('INTERMEDIATE_REGION_ID', 'int'),
 ('ORGANIZATION_REGION_ID', 'int')]

In [0]:
# Reading the countries file into a new variable without specifying the schema 
countries_dt = spark.read.csv(path=countries_path, header=True)

In [0]:
# All data types are all string
countries_dt.dtypes

Out[6]: [('COUNTRY_ID', 'string'),
 ('NAME', 'string'),
 ('NATIONALITY', 'string'),
 ('COUNTRY_CODE', 'string'),
 ('ISO_ALPHA2', 'string'),
 ('CAPITAL', 'string'),
 ('POPULATION', 'string'),
 ('AREA_KM2', 'string'),
 ('REGION_ID', 'string'),
 ('SUB_REGION_ID', 'string'),
 ('INTERMEDIATE_REGION_ID', 'string'),
 ('ORGANIZATION_REGION_ID', 'string')]

In [0]:
# Using the cast method to cast the population column as IntegerType(), IntegerType() has already been imported in the first cell when creating the schema
countries_dt.select(countries_dt['population'].cast(IntegerType())).dtypes

Out[7]: [('population', 'int')]

In [0]:
# Using the cast method to cast the population column as StringType(), StringType() has already been imported in the first cell when creating the schema
countries.select(countries['population'].cast(StringType())).dtypes

Out[8]: [('population', 'string')]

#### Math Functions
* Math Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#math-functions

In [0]:
# Simple arithmetic to return the population in milions
countries.select(countries['population']/1000000).withColumnRenamed('(population / 1000000)','population_m').limit(5).display()

# Adding the column to a variable
countries_2 = countries.select(countries['population']/1000000).withColumnRenamed('(population / 1000000)','population_m')

# Using the round function to round to 2 decimal places
from pyspark.sql.functions import round
countries_2.select(round(countries_2['population_m'],2)).withColumnRenamed('round(population_m, 2)', 'population_m').limit(5).display()

population_m
38.041754
2.880917
43.053054
0.055312
0.077142


population_m
38.04
2.88
43.05
0.06
0.08


In [0]:
# create new column population_m_r1 rounded to 1 decimal in countries df
countries.withColumn('population_m_r1', round(countries['population']/1000000, 1)).limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID,population_m_r1
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30,38.0
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20,2.9
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20,43.1
4,American Samoa,American Samoan,ASM,AS,Pago Pago,55312.0,199,40,20,null,30,0.1
5,Andorra,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20,0.1


####A bit of sorting

In [0]:
#importing ascending and descending
from pyspark.sql.functions import asc, desc

In [0]:
countries.sort(countries['population'].asc()).limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
97,Heard Island and McDonald Islands,Heard Island or McDonald Islands,HMD,HM,null,0.0,368,40,90,null,30
30,Bouvet Island,Bouvet Island,BVT,BV,null,0.0,49,10,10,40,40
206,South Georgia and the South Sandwich Islands,South Georgia or South Sandwich Islands,SGS,GS,Grytviken,30.0,3903,10,10,40,40
175,Pitcairn,Pitcairn Island,PCN,PN,Adamstown,68.0,43,40,20,null,30
79,French Southern Territories,French Southern Territories,ATF,TF,Saint Pierre,150.0,439780,50,160,50,20


In [0]:
countries.sort(countries['population'].desc_nulls_last()).limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID
45,China,Chinese,CHN,CN,Beijing,1.433783686E9,9706961,30,60,null,30
103,India,Indian,IND,IN,New Delhi,1.366417754E9,3287590,30,30,null,30
236,United States of America,American,USA,US,"Washington, D.C.",3.29064917E8,9372610,10,80,null,10
104,Indonesia,Indonesian,IDN,ID,Jakarta,2.70625568E8,1904569,30,100,null,30
167,Pakistan,Pakistani,PAK,PK,Islamabad,2.16565318E8,881912,30,30,null,30


#### String Functions
* https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#string-functions

In [0]:
#to import all functions
from pyspark.sql.functions import *

In [0]:
# Use concat_ws to concatenate columns
countries.select(concat_ws('-', countries['name'], lower(countries['country_code'])).alias('Country-country_code')).limit(5).display()

Country-country_code
Afghanistan-afg
Albania-alb
Algeria-dza
American Samoa-asm
Andorra-and


#### Datetime Functions
* Datetime Functions: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html#datetime-functions
* Datetime Patterns: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [0]:
#using current_timestamp and withColumn to add column to df
countries = countries.withColumn('timestamp', current_timestamp())
countries.limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID,timestamp
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30,2023-05-08T06:43:32.328+0000
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20,2023-05-08T06:43:32.328+0000
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20,2023-05-08T06:43:32.328+0000
4,American Samoa,American Samoan,ASM,AS,Pago Pago,55312.0,199,40,20,null,30,2023-05-08T06:43:32.328+0000
5,Andorra,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20,2023-05-08T06:43:32.328+0000


In [0]:
# Use year to extract the year
countries.select(year(countries['timestamp'])).limit(5).display()

year(timestamp)
2023
2023
2023
2023
2023


In [0]:
#adding literal value to a column in order to convert the data type from string to date
countries = countries.withColumn('date_lit', lit('08-05-2023'))
countries.limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID,timestamp,date_lit
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30,2023-05-08T06:47:33.759+0000,08-05-2023
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20,2023-05-08T06:47:33.759+0000,08-05-2023
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20,2023-05-08T06:47:33.759+0000,08-05-2023
4,American Samoa,American Samoan,ASM,AS,Pago Pago,55312.0,199,40,20,null,30,2023-05-08T06:47:33.759+0000,08-05-2023
5,Andorra,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20,2023-05-08T06:47:33.759+0000,08-05-2023


In [0]:
# Use the to_date function to convert the string to a date
countries = countries.withColumn('conv_date', to_date(countries['date_lit'],'dd-MM-yyyy'))
countries.limit(5).display()

COUNTRY_ID,NAME,NATIONALITY,COUNTRY_CODE,ISO_ALPHA2,CAPITAL,POPULATION,AREA_KM2,REGION_ID,SUB_REGION_ID,INTERMEDIATE_REGION_ID,ORGANIZATION_REGION_ID,timestamp,date_lit,conv_date
1,Afghanistan,Afghan,AFG,AF,Kabul,3.8041754E7,652230,30,30,null,30,2023-05-08T06:53:19.604+0000,08-05-2023,2023-05-08
2,Albania,Albanian,ALB,AL,Tirana,2880917.0,28748,20,70,null,20,2023-05-08T06:53:19.604+0000,08-05-2023,2023-05-08
3,Algeria,Algerian,DZA,DZ,Algiers,4.3053054E7,2381741,50,40,null,20,2023-05-08T06:53:19.604+0000,08-05-2023,2023-05-08
4,American Samoa,American Samoan,ASM,AS,Pago Pago,55312.0,199,40,20,null,30,2023-05-08T06:53:19.604+0000,08-05-2023,2023-05-08
5,Andorra,Andorran,AND,AD,Andorra la Vella,77142.0,468,20,70,null,20,2023-05-08T06:53:19.604+0000,08-05-2023,2023-05-08
